In [ ]:
pip install lightautoml

In [ ]:
pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import typing
import torch
from google.colab import drive
from sklearn.model_selection import train_test_split
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.automl.presets.tabular_presets import TabularUtilizedAutoML
from lightautoml.tasks import Task
from google.colab import files
from sklearn.model_selection import KFold
drive.mount('/content/drive')

In [21]:
# кастомная метрика от райфа
THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1
def deviation_metric_vec(y_true: np.array, y_pred: np.array) -> float:
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    
    metr = deviation * 0.0 + 9
    
    metr[np.abs(deviation) <= THRESHOLD] = 0
    
    metr[deviation <= - 4 * THRESHOLD] = 9 * NEGATIVE_WEIGHT
    
    mask = (-4 * THRESHOLD < deviation) & (deviation < -THRESHOLD)
    metr[mask] = NEGATIVE_WEIGHT * ((deviation[mask] / THRESHOLD) + 1) ** 2
    
    mask = (deviation < 4 * THRESHOLD) & (deviation > THRESHOLD)
    metr[mask] = ((deviation[mask] / THRESHOLD) - 1) ** 2
    
    return metr.mean()

In [8]:
train_data = pd.read_csv('/content/drive/My Drive/data/train.csv', low_memory=False)
test_data = pd.read_csv('/content/drive/My Drive/data/test.csv', low_memory=False)
train_data.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,139937.500000,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,60410.714286,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,Ярославль,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,45164.761264,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,Новокузнецк,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,28805.263158,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,Химки,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,13222.591362,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [9]:
train_data.shape, test_data.shape

((279792, 77), (2974, 76))

In [10]:
for col in train_data.select_dtypes(include=np.number).columns:
    train_data[col] = pd.to_numeric(train_data[col], downcast = 'unsigned')

In [11]:
# добавляем новые фичи
city_population = pd.read_csv('/content/drive/My Drive/data/city_population.csv')
zarplaty = pd.read_excel('/content/drive/My Drive/data/zarplaty.xlsx')

def city_type(row):
    if row >=1000000:
        return "1Million"
    elif  (row<1000000)&(row >200000):
        return "Medium"
    elif  (row <=200000):
        return "Small"
    
def add_features(df):
    df['age'] = round(2021 - df['reform_mean_year_building_500'])
    df.city = df.city.apply(lambda x: x.lower())
    
    city_population_clean = city_population.groupby('settlement').agg({'population':'sum'}).reset_index()
    city_population_clean.columns = ['city', 'city_population']
    city_population_clean['city_population']
    city_population_clean.city = city_population_clean.city.apply(lambda x: x.lower())
    df = df.merge(city_population_clean, on = 'city', how='left')
    
    for col in df.select_dtypes(include=np.number).columns:
        df[col] = pd.to_numeric(df[col], downcast = 'unsigned')
    
    df['city_type'] = df['city_population'].apply(lambda x: city_type(x))
    df.loc[df.city  == 'москва', 'city_type'] = "Capital"
    df.loc[df.city  == 'санкт-Петербург', 'city_type'] = "Capital"
    
    df = df.merge(zarplaty, on = 'region', how='left')
    df['zarplata'] = pd.to_numeric(df['zarplata'], downcast = 'unsigned')    
    return df

In [12]:
train_data = add_features(train_data)
test_data = add_features(test_data)

In [16]:
# оставляем только данные с ручной оценкой
train_data = train_data.query('price_type == 1')
data = pd.concat([train_data, test_data], ignore_index = False)

In [17]:
labeled_features = ['city','osm_city_nearest_name', 'region','city_type']

for col in labeled_features:
    data[col].fillna('UNKNOWN', inplace=True)

data = data.drop('date', axis=1)
data = data.drop('id', axis=1)
data = data.drop(['street'], axis=1)

# вручную обрабатываем этажи (можно добавить количество этажей как фичу)
data['floor'] = data['floor'].mask(data['floor'] == '-1.0', -1) \
              .mask(data['floor'] == '-2.0', -2) \
              .mask(data['floor'] == '-3.0', -3) \
              .mask(data['floor'] == 'подвал, 1', 1) \
              .mask(data['floor'] == 'подвал', -1) \
              .mask(data['floor'] == 'цоколь, 1', 1) \
              .mask(data['floor'] == '1,2,антресоль', 1) \
              .mask(data['floor'] == 'цоколь', 0) \
              .mask(data['floor'] == 'тех.этаж (6)', 6) \
              .mask(data['floor'] == 'Подвал', -1) \
              .mask(data['floor'] == 'Цоколь', 0) \
              .mask(data['floor'] == 'фактически на уровне 1 этажа', 1) \
              .mask(data['floor'] == '1,2,3', 1) \
              .mask(data['floor'] == '1, подвал', 1) \
              .mask(data['floor'] == '1,2,3,4', 1) \
              .mask(data['floor'] == '1,2', 1) \
              .mask(data['floor'] == '1,2,3,4,5', 1) \
              .mask(data['floor'] == '5, мансарда', 5) \
              .mask(data['floor'] == '1-й, подвал', 1) \
              .mask(data['floor'] == '1, подвал, антресоль', 1) \
              .mask(data['floor'] == 'мезонин', 2) \
              .mask(data['floor'] == 'подвал, 1-3', 1) \
              .mask(data['floor'] == '1 (Цокольный этаж)', 0) \
              .mask(data['floor'] == '3, Мансарда (4 эт)', 3) \
              .mask(data['floor'] == 'подвал,1', 1) \
              .mask(data['floor'] == '1, антресоль', 1) \
              .mask(data['floor'] == '1-3', 1) \
              .mask(data['floor'] == 'мансарда (4эт)', 4) \
              .mask(data['floor'] == '1, 2.', 1) \
              .mask(data['floor'] == 'подвал , 1 ', 1) \
              .mask(data['floor'] == '1, 2', 1) \
              .mask(data['floor'] == 'подвал, 1,2,3', 1) \
              .mask(data['floor'] == '1 + подвал (без отделки)', 1) \
              .mask(data['floor'] == 'мансарда', 3) \
              .mask(data['floor'] == '2,3', 2) \
              .mask(data['floor'] == '4, 5', 4) \
              .mask(data['floor'] == '1-й, 2-й', 1) \
              .mask(data['floor'] == '1 этаж, подвал', 1) \
              .mask(data['floor'] == '1, цоколь', 1) \
              .mask(data['floor'] == 'подвал, 1-7, техэтаж', 1) \
              .mask(data['floor'] == '3 (антресоль)', 3) \
              .mask(data['floor'] == '1, 2, 3', 1) \
              .mask(data['floor'] == 'Цоколь, 1,2(мансарда)', 1) \
              .mask(data['floor'] == 'подвал, 3. 4 этаж', 3) \
              .mask(data['floor'] == 'подвал, 1-4 этаж', 1) \
              .mask(data['floor'] == 'подва, 1.2 этаж', 1) \
              .mask(data['floor'] == '2, 3', 2) \
              .mask(data['floor'] == '7,8', 7) \
              .mask(data['floor'] == '1 этаж', 1) \
              .mask(data['floor'] == '1-й', 1) \
              .mask(data['floor'] == '3 этаж', 3) \
              .mask(data['floor'] == '4 этаж', 4) \
              .mask(data['floor'] == '5 этаж', 5) \
              .mask(data['floor'] == 'подвал,1,2,3,4,5', 1) \
              .mask(data['floor'] == 'подвал, цоколь, 1 этаж', 1) \
              .mask(data['floor'] == '3, мансарда', 3) \
              .mask(data['floor'] == 'цоколь, 1, 2,3,4,5,6', 1) \
              .mask(data['floor'] == ' 1, 2, Антресоль', 1) \
              .mask(data['floor'] == '3 этаж, мансарда (4 этаж)', 3) \
              .mask(data['floor'] == 'цокольный', 0) \
              .mask(data['floor'] == '1,2 ', 1) \
              .mask(data['floor'] == '3,4', 3) \
              .mask(data['floor'] == 'подвал, 1 и 4 этаж', 1) \
              .mask(data['floor'] == '5(мансарда)', 5) \
              .mask(data['floor'] == 'технический этаж,5,6', 5) \
              .mask(data['floor'] == ' 1-2, подвальный', 1) \
              .mask(data['floor'] == '1, 2, 3, мансардный', 1) \
              .mask(data['floor'] == 'подвал, 1, 2, 3', 1) \
              .mask(data['floor'] == '1,2,3, антресоль, технический этаж', 1) \
              .mask(data['floor'] == '3, 4', 3) \
              .mask(data['floor'] == '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)', 1) \
              .mask(data['floor'] == '1,2,3,4, подвал', 1) \
              .mask(data['floor'] == '2-й', 2) \
              .mask(data['floor'] == '1, 2 этаж', 1) \
              .mask(data['floor'] == 'подвал, 1, 2', 1) \
              .mask(data['floor'] == '1-7', 1) \
              .mask(data['floor'] == '1 (по док-м цоколь)', 1) \
              .mask(data['floor'] == '1,2,подвал ', 1) \
              .mask(data['floor'] == 'подвал, 2', 2) \
              .mask(data['floor'] == 'подвал,1,2,3', 1) \
              .mask(data['floor'] == '1,2,3 этаж, подвал ', 1) \
              .mask(data['floor'] == '1,2,3 этаж, подвал', 1) \
              .mask(data['floor'] == '2, 3, 4, тех.этаж', 2) \
              .mask(data['floor'] == 'цокольный, 1,2', 1) \
              .mask(data['floor'] == 'Техническое подполье', -1) \
              .mask(data['floor'] == '1.2', 1) \
              .astype(float)
data = pd.get_dummies(data)
data

,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,...,region_Кемеровская область,region_Кировская область,region_Коми,region_Костромская область,region_Краснодарский край,region_Красноярский край,region_Курская область,region_Ленинградская область,region_Липецкая область,region_Мордовия,region_Москва,region_Московская область,region_Нижегородская область,region_Новосибирская область,region_Омская область,region_Орловская область,region_Пензенская область,region_Пермский край,region_Приморский край,region_Ростовская область,region_Самарская область,region_Санкт-Петербург,region_Саратовская область,region_Свердловская область,region_Смоленская область,region_Ставропольский край,region_Татарстан,region_Томская область,region_Тульская область,region_Тюменская область,region_Удмуртия,region_Ульяновская область,region_Ханты-Мансийский АО,region_Челябинская область,region_Ярославская область,city_type_1Million,city_type_Capital,city_type_Medium,city_type_Small,city_type_UNKNOWN
275474,NaN,56.063615,92.958428,0,7,14,26,0,0,0,0,0,0,0,2,7.795659,1095286.0,0.041323,2,6,25,35,0,0,0,2,0,1,2,3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
275475,NaN,51.534581,46.020549,13,198,345,462,0,0,0,0,4,44,66,89,0.987160,842097.0,0.144936,0,4,34,59,0,1,6,9,0,13,22,27,7,17,27,4,7,9,2,5,7,11,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
275476,NaN,56.026884,92.818323,3,15,23,33,0,0,0,0,0,1,3,4,3.730568,1095286.0,0.141526,0,16,35,56,0,0,0,0,2,3,3,3,0,1,1,3,3,3,0,0,1,2,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
275477,NaN,52.275528,104.251444,0,10,26,40,0,1,1,1,0,2,6,8,2.344310,623562.0,0.281491,0,6,6,6,0,0,0,0,0,0,0,1,0,4,4,0,0,0,0,1,2,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
275478,NaN,50.576545,36.584197,4,48,73,92,0,0,6,6,1,7,11,15,2.170184,384425.0,0.065996,1,19,28,46,1,5,10,11,0,2,2,4,3,3,4,2,2,2,0,2,2,10,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969,4.0,55.978180,92.891457,0,3,6,16,0,0,0,0,0,1,1,1,3.572596,1095286.0,0.077641,1,15,44,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2970,1.0,56.459183,84.979872,2,33,111,222,0,0,1,1,1,9,24,45,3.614641,576624.0,0.116917,0,13,32,62,0,1,1,2,0,1,3,8,0,2,4,1,2,6,2,4,6,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2971,-1.0,54.523247,36.295168,2,25,54,99,0,0,0,2,1,2,8,12,2.573999,342936.0,0.016371,8,22,41,77,0,1,2,2,0,0,0,2,2,3,4,4,4,7,0,0,0,1,...,0,0,0,0,0,

In [18]:
train_data = data.iloc[0:train_data.shape[0]]
test_data = data.iloc[train_data.shape[0]:]

In [19]:
train_data.shape, test_data.shape

((4493, 485), (2974, 485))

In [20]:
target = train_data['per_square_meter_price']
train_data = train_data.drop('per_square_meter_price', axis=1)
test_data = test_data.drop('per_square_meter_price', axis=1)

In [ ]:
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
sampler = TPESampler(seed=13)

def create_model(trial):
    num_leaves = trial.suggest_int("num_leaves", 2, 1500)
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_child_samples = trial.suggest_int('min_child_samples', 2, 3000)
    learning_rate = trial.suggest_uniform('learning_rate', 0.001, 0.3)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 2, 300)
    feature_fraction = trial.suggest_uniform('feature_fraction', 0.00001, 1.0)
    
    model = lgbm.LGBMRegressor(
        num_leaves=num_leaves,
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        min_data_in_leaf=min_data_in_leaf,
        learning_rate=learning_rate,
        feature_fraction=feature_fraction,
        random_state=13,
        n_jobs=-1
)
    return model

def objective(trial):
    model = create_model(trial)
    X_train = train_data.iloc[:train_data.shape[0]-1125]
    X_val = train_data.iloc[train_data.shape[0]-1125:]
    y_train = target.iloc[:train_data.shape[0]-1125]
    y_val = target[train_data.shape[0]-1125:]
    model.fit(X_train.values, y_train,)
    result = model.predict(X_val.values)
    score = deviation_metric_vec(y_val, result)
    return score

study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=100)
params_lgbm = study.best_params

In [ ]:
model = lgbm.LGBMRegressor(**params_lgbm)
kFold_random_state = [13, 666]
n_splits = 5

final_loss = list()

submission = pd.read_csv('/content/drive/My Drive/data/test_submission.csv')
submission.iloc[:, 1] = 0


for ind_k, random_state in enumerate(kFold_random_state):
    kFold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    total_loss = list()

    for iteration, (train_index, valid_index) in enumerate(kFold.split(train_data, target)):

        X_train, X_valid = train_data.iloc[train_index].reset_index(drop=True), train_data.iloc[valid_index].reset_index(drop=True)
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]

        model.fit(X_train.values, y_train)
        valid_pred = model.predict(X_valid.values)
        loss = deviation_metric_vec(y_valid, valid_pred)

        predict = model.predict(test_data)
        submission['per_square_meter_price'] = submission['per_square_meter_price'] + predict / 10

        total_loss.append(np.mean(loss))

    final_loss.append(np.mean(total_loss))
    print(f'deviation_metric: {np.mean(total_loss)}')
print(f'Final deviation_metric: {np.mean(final_loss)}')

In [292]:
# подрежем таргеты сверху
submission['per_square_meter_price'] = submission['per_square_meter_price'] * 0.9

submission.loc[submission['per_square_meter_price'] >= 200000, 'per_square_meter_price'] \
    = submission.loc[submission['per_square_meter_price'] >= 200000, 'per_square_meter_price'] * 0.9
pred = submission['per_square_meter_price']
print(pred)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [317]:
# сделаем пседво лэйблы
test_data['per_square_meter_price'] = list(pred)
train_data['per_square_meter_price'] = target
df = pd.concat([train_data, test_data], ignore_index = False)
test_data = test_data.drop(columns=['per_square_meter_price'], axis=1)

In [232]:
test_data['per_square_meter_price'] = pred
train_data['per_square_meter_price'] = target
df = pd.concat([train_data, test_data], ignore_index = False)
test_data = test_data.drop('per_square_meter_price', axis=1)
df.shape, test_data.shape

((7467, 482), (2974, 481))

In [ ]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TIMEOUT = 500
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

task = Task('reg', loss = 'mae', metric = deviation_metric_vec)
roles = {'target': 'per_square_meter_price'}

automl = TabularAutoML(task = task,
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               reader_params = {'n_jobs': N_THREADS,
                                                'cv': N_FOLDS,
                                                'random_state': RANDOM_STATE})

oof_pred = automl.fit_predict(df, roles = roles)

In [319]:
test_pred = automl.predict(test_data)
output = pd.read_csv('/content/drive/My Drive/data/test_submission.csv')
output["per_square_meter_price"] = test_pred.data[:, 0]
submission.to_csv('sub.csv', index=False)
files.download("sub.csv")